In [31]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [32]:
# Загрузка данных (замените 'path_to_your_data.csv' на путь к вашему датасету)
df = pd.read_csv("/content/primary_results.csv")

In [33]:
# Предпросмотр данных
print(df.head())

# Проверка на наличие пропусков по колонкам
print(df.isnull().sum())

     state state_abbreviation   county    fips     party        candidate  \
0  Alabama                 AL  Autauga  1001.0  Democrat   Bernie Sanders   
1  Alabama                 AL  Autauga  1001.0  Democrat  Hillary Clinton   
2  Alabama                 AL  Baldwin  1003.0  Democrat   Bernie Sanders   
3  Alabama                 AL  Baldwin  1003.0  Democrat  Hillary Clinton   
4  Alabama                 AL  Barbour  1005.0  Democrat   Bernie Sanders   

   votes  fraction_votes  
0    544           0.182  
1   2387           0.800  
2   2694           0.329  
3   5290           0.647  
4    222           0.078  
state                   0
state_abbreviation      0
county                  0
fips                  100
party                   0
candidate               0
votes                   0
fraction_votes          0
dtype: int64


In [34]:
# Создаем список категориальных столбцов (в датасете все столбцы, кроме 'fips', 'votes', 'fraction_votes', можно рассматривать как категориальные)
categorical_cols = ['state', 'state_abbreviation', 'county', 'party', 'candidate']

# Список числовых столбцов (исключим 'fips', т.к. это уникальный код и не несет полезной информации для модели)
numerical_cols = ['votes', 'fraction_votes']

# Обработка пропусков в данных
# Для категориальных признаков заменим пропуски на слово 'missing'
# Для числовых - на среднее значение (можно использовать другие стратегии в зависимости от задачи и распределения признаков)
imputer_categorical = SimpleImputer(strategy='constant', fill_value='missing')
imputer_numerical = SimpleImputer(strategy='mean')

In [35]:
# Кодирование категориальных признаков
encoder = OneHotEncoder(handle_unknown='ignore')

# Масштабирование данных
scaler = StandardScaler()

In [38]:
# Создание ColumnTransformer, чтобы выполнить необходимые преобразования для различных типов столбцов
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', Pipeline([('imputer', imputer_categorical), ('encoder', encoder)]), categorical_cols),
        ('num', Pipeline([('imputer', imputer_numerical), ('scaler', scaler)]), numerical_cols)
    ])

# Применяем обработку и преобразования к нашим данным
df_processed = preprocessor.fit_transform(df)

# Преобразовываем результат в dense формат, если он в sparse
if hasattr(df_processed, "toarray"):
    df_processed = df_processed.toarray()

# Получаем имена новых признаков после OneHotEncoding
# Важно убедиться, что индексация трансформеров правильная
ohe_feature_names = preprocessor.named_transformers_['cat']['encoder'].get_feature_names_out(categorical_cols)
feature_names = list(ohe_feature_names) + numerical_cols

# Конвертируем обработанные данные обратно в pandas DataFrame для удобства
df_processed = pd.DataFrame(df_processed, columns=feature_names)

# Предпросмотр обработанных данных
print(df_processed.head())

# В данном коде мы использовали SimpleImputer для обработки пропусков в данных,
# OneHotEncoder для кодирования категориальных признаков и StandardScaler для масштабирования числовых данных.
# Все шаги были объединены в ColumnTransformer, что позволяет применять различные преобразования
# к определенным столбцам данных.

   state_Alabama  state_Alaska  state_Arizona  state_Arkansas  \
0            1.0           0.0            0.0             0.0   
1            1.0           0.0            0.0             0.0   
2            1.0           0.0            0.0             0.0   
3            1.0           0.0            0.0             0.0   
4            1.0           0.0            0.0             0.0   

   state_California  state_Colorado  state_Connecticut  state_Delaware  \
0               0.0             0.0                0.0             0.0   
1               0.0             0.0                0.0             0.0   
2               0.0             0.0                0.0             0.0   
3               0.0             0.0                0.0             0.0   
4               0.0             0.0                0.0             0.0   

   state_Florida  state_Georgia  ...  candidate_Jeb Bush  \
0            0.0            0.0  ...                 0.0   
1            0.0            0.0  ...        